# dataset preparing
1. download data.zip to project root folder
2. move data.zip and unzip in the code below.

In [ ]:
import os

os.makedirs("./data/coco/fashion", exist_ok=True)
os.rename("./data.zip", "./data/coco/fashion/data.zip")

!unzip ./data/coco/fashion/data.zip -d ./data/coco/fashion

# split validation dataset
- 학습 데이터셋의 20%를 validation dataset으로 사용

In [ ]:
import os
import json

coco_file = './data/coco/fashion/train.json'
output_dir = './data/coco/fashion'
train_ratio = 0.8

def save_coco(file_path, anno_dict, image_dict, categories, image_id_list):
    coco = {'categories': categories}
    images = []
    annotations = []
    for image_id in image_id_list:
        images.append(image_dict[image_id])
        if image_id not in anno_dict:
            print("skip! image id  not in anno dict. image_id: {}".format(image_id))
            continue
        for anno in anno_dict[image_id]:
            annotations.append(anno)

    coco['annotations'] = annotations
    coco['images'] = images
    json.dump(coco, open(file_path, "w+"))
    
os.makedirs(output_dir, exist_ok=True)

print("loading coco dataset")
coco = json.load(open(coco_file))
print("loaded")

images = coco['images']
annotations = coco['annotations']
categories = coco['categories']

image_id_list = []
image_dict = {}
for image in images:
    image_id = image['id']
    image_id_list.append(image_id)
    image_dict[image_id] = image

anno_dict = {}
for anno in annotations:
    image_id = anno['image_id']
    if image_id not in anno_dict:
        anno_dict[image_id] = []
    anno_dict[image_id].append(anno)

split_idx = round(len(image_id_list) * train_ratio)
train_image_id_list = image_id_list[:split_idx]
val_image_id_list = image_id_list[split_idx:]

save_coco(os.path.join(output_dir, "train_split.json"), anno_dict, image_dict, categories, train_image_id_list)
save_coco(os.path.join(output_dir, "val_split.json"), anno_dict, image_dict, categories, val_image_id_list)

print("done")

# move train & test images to each folder. 

In [ ]:
import glob
import shutil

def move_images_to_dir(image_root_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    image_dirs = glob.glob(os.path.join(image_root_dir, "*"))

    for i, image_dir in enumerate(image_dirs):
        image_files = glob.glob(os.path.join(image_dir, "*"))
        for j, image_path in enumerate(image_files):
            if j % 100 == 0:
                print("moving dir {}/{}, files {}/{}".format(i, len(image_dirs), j, len(image_files)))
            shutil.move(image_path, output_dir)
    print("done")

move_images_to_dir("./data/coco/fashion/train", "./data/coco/fashion/train_images")
move_images_to_dir("./data/coco/fashion/test", "./data/coco/fashion/test_images")

# Training
- Detection Model: DetectoRS
- backbone: Resnet50
- 4 steps training(Transfer learning from pretrained model with imagenet and 3times FineTunings)

# Training 1 Transfer Learning
- image scale: 1333, 800
- trained with splited traininig data
- optimizer = dict(type='SGD', lr=0.02, momentum=0.9, weight_decay=0.0001)
- training epoch: 12epoch
- transfer learning from pretrained torchvision://resnet50 with imagenet

In [ ]:
!CUDA_VISIBLE_DEVICES=0 \
python tools/train.py \
configs/fashion_detectors_tr1.py

# multiple gpus training
#! OMP_NUM_THREADS=1 CUDA_VISIBLE_DEVICES=0,1 \
#bash tools/dist_train.sh \
#configs/fashion_detectors_tr1.py 2

# Training 2 FineTuning
- image scale: 800, 800
- trained with all traininig data
- finetuning from work_dirs/fashion_detectors/epoch_12.pth
- optimizer = dict(type='SGD', lr=0.02, momentum=0.9, weight_decay=0.0001)
- training epoch: from 12epoch to 24epoch

In [ ]:
!wget -nc https://font-recognizer-bucket.s3.us-east-2.amazonaws.com/resource/kaggle/fashion_mmdetection/epoch_12.pth -P ./work_dirs/fashion_detectors/

!CUDA_VISIBLE_DEVICES=0 \
python tools/train.py \
configs/fashion_detectors_tr2.py

# Training 3 FineTuning
- image scale: 800, 800
- trained with all traininig data
- finetuning from work_dirs/fashion_detectors/epoch_24.pth
- optimizer = dict(type='SGD', lr=0.004, momentum=0.9, weight_decay=0.0001)
- training epoch: from 24epcoh to 36epoch

In [ ]:
!wget -nc https://font-recognizer-bucket.s3.us-east-2.amazonaws.com/resource/kaggle/fashion_mmdetection/epoch_24.pth -P ./work_dirs/fashion_detectors/

!CUDA_VISIBLE_DEVICES=0 \
python tools/train.py \
configs/fashion_detectors_tr3.py

# Training 4 FineTuning
- image scale: 800, 800
- trained with all traininig data
- finetuning from work_dirs/fashion_detectors/epoch_34.pth
- optimizer = dict(type='SGD', lr=0.002, momentum=0.9, weight_decay=0.0001)
- training epoch: from 34epoch to 50epoch

In [ ]:
!wget -nc https://font-recognizer-bucket.s3.us-east-2.amazonaws.com/resource/kaggle/fashion_mmdetection/epoch_34.pth -P ./work_dirs/fashion_detectors/

!CUDA_VISIBLE_DEVICES=0 \
python tools/train.py \
configs/fashion_detectors_tr4.py

# Test 2 models
1. epoch 34 model (top 1 model)
2. epoch 50 model (top 2 model)

- soft_nms
- score_thr: 0.81

- 리더보드 기준 약 7.8% 향상

In [ ]:
# 테스트용 파라미터 (실행할 필요 없음)
test_cfg = dict(
    rpn=dict(
        nms_across_levels=False,
        nms_pre=1000,
        nms_post=1000,
        max_num=1000,
        nms_thr=0.7,
        min_bbox_size=0),
    rcnn=dict(
        score_thr=0.81,
        nms=dict(type='soft_nms', iou_threshold=0.5),
        max_per_img=100,
        mask_thr_binary=0.5)
)

In [ ]:
!wget -nc https://font-recognizer-bucket.s3.us-east-2.amazonaws.com/resource/kaggle/fashion_mmdetection/epoch_34.pth -P ./work_dirs/fashion_detectors/
!wget -nc https://font-recognizer-bucket.s3.us-east-2.amazonaws.com/resource/kaggle/fashion_mmdetection/epoch_50.pth -P ./work_dirs/fashion_detectors/
    
!CUDA_VISIBLE_DEVICES=0 \
python tools/test.py \
configs/fashion_detectors_test.py \
work_dirs/fashion_detectors/epoch_34.pth \
--format-only \
--eval-options "jsonfile_prefix=./work_dirs/fashion_detectors/e34"

!CUDA_VISIBLE_DEVICES=0 \
python tools/test.py \
configs/fashion_detectors_test.py \
work_dirs/fashion_detectors/epoch_50.pth \
--format-only \
--eval-options "jsonfile_prefix=./work_dirs/fashion_detectors/e50"

# multiple gpus training
# ! OMP_NUM_THREADS=1 CUDA_VISIBLE_DEVICES=0,1 \
# bash tools/dist_test.sh \
# configs/fashion_detectors_test.py work_dirs/fashion_detectors/epoch_34.pth 2 --format-only --eval-options "jsonfile_prefix=./work_dirs/fashion_detectors/e34"

# ! OMP_NUM_THREADS=1 CUDA_VISIBLE_DEVICES=0,1 \
# bash tools/dist_test.sh \
# configs/fashion_detectors_test.py work_dirs/fashion_detectors/epoch_50.pth 2 --format-only --eval-options "jsonfile_prefix=./work_dirs/fashion_detectors/e50"

# Ensemble 2 models, mask post processing
- 각 모델의 테스트 결과인 마스크들을 서로 비교하여 새로운 마스크를 추가하거나 동일한 마스크를 머지하는 과정
- 앙상블 하려는 모든 모델의 마스크들을 이미지 별로 취합 후, 이미지에 mask가 한개만 있으면 무조건 mask를 결과에 포함시킨다.
- 이미지에 mask가 여러개 있으면 모든 마스크끼리 iou를 비교하여 iou가 0.7이상일 경우에는 2개의 마스크를 merge(단순 더하기)하고,
- 첫번째로 입력한 모델의 카테고리로 결과에 포함시킨다.
- merge되지 않는 마스크들은 그대로 결과에 포함시킨다.
- 이 단순한 로직으로 리더보드 기준 약 1% 향상

In [ ]:
!python -m demo.ensemble_submission_parallel \
--result_files=work_dirs/fashion_detectors/e34.segm.json,work_dirs/fashion_detectors/e50.segm.json \
--output_dir=work_dirs/fashion_detectors/ens_e34_e50_softnms_scorethr081_ens_iouthr07 \
--iou_thr=0.7 \
--use_merge --n=10

!head -n 2 work_dirs/fashion_detectors/ens_e34_e50_softnms_scorethr081_ens_iouthr07/submission.csv

# 최종 제출 파일 경로
work_dirs/fashion_detectors/ens_e34_e50_softnms_scorethr081_ens_iouthr07/submission.csv

In [ ]:
!ls -l work_dirs/fashion_detectors/ens_e34_e50_softnms_scorethr081_ens_iouthr07/submission.csv